In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model = "gpt-4.1-nano-2025-04-14")

## RunnablePassthrough
* It does not do anything to the input data.
* Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

In [3]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [4]:
out = chain.invoke('Rohit')
out

'Rohit'

In [6]:
type('Rohit')

str

In [5]:
type(out)

str

## RunnableLambda
* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
* Let's define a very simple function to create Russian lastnames:

In [7]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [9]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [10]:
chain.invoke('Rohit')

'Rohitovich'

## RunnableParallel
* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
    * operation_a will use RunnablePassthrough.
    * operation_b will use RunnableLambda with the russian_lastname function.

In [11]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [12]:
chain.invoke("Rohit")

{'operation_a': 'Rohit', 'operation_b': 'Rohitovich'}

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [14]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [15]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [16]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | chatModel | output_parser

In [17]:
chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})

'One curious fact about Roman Abramovich is that he holds several citizenships beyond Russian—he also has Israeli and Portuguese nationality. This diverse citizenship portfolio is relatively uncommon among Russian oligarchs and reflects his strategic international connections.'

## Let's see a more advanced use of RunnableParallel

In [18]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["DataSage with Rohit focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-4.1-nano-2025-04-14")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("What is DataSage with Rohit?")

'DataSage with Rohit is a platform that provides content on Data Science, AI, Machine Learning, Deep Learning, Computer Vision, NLP, Python programming, and related topics in English.'

In [19]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

model = ChatOpenAI(model="gpt-4.1-nano-2025-04-14")

vectorstore = FAISS.from_texts(
    ["DataSage with Rohit focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "What is DataSage with Rohit?", "language": "Pirate English"})

"Arrr matey! DataSage with Rohit be a treasure trove o' content on Data Science, AI, ML, DL, CV, NLP, and Python programming alls in English, aye!"